In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

df = pd.read_csv(r"C:\ise3230\grocery_kroger_priced_2.csv")

items = df["Product_Name"].tolist()

fat = df.set_index("Product_Name")["Fat_g"].to_dict()
sodium = df.set_index("Product_Name")["Sodium_g"].to_dict()
protein = df.set_index("Product_Name")["Protein_g"].to_dict()
calories = df.set_index("Product_Name")["Calories"].to_dict()
cost = df.set_index("Product_Name")["Price"].to_dict()

m = gp.Model("diet_7day")
m.Params.LogToConsole = 0

# Want to create a weekly meal plan over the course of 7 school days
days = range(7)

# x[i, d] = number of servings of food i on day d. We constrain it to be at most 2
x = m.addVars(items, days, vtype=GRB.INTEGER, lb=0, ub=2, name="x")

# y[i, d] = 1 if food i is used at least once on day d
y = m.addVars(items, days, vtype=GRB.BINARY, name="y")

# Want to minimize the cost of the meal plan
m.setObjective(gp.quicksum(cost[i] * x[i, d] for i in items for d in days), GRB.MINIMIZE)

FAT_MAX = 275
SODIUM_MAX = 6
PROTEIN_MIN = 150
CALORIE_MIN = 10000
#FIBER_MIN = 75
#SUGAR_MAX = 100
MAX_ITEMS_PER_DAY = 30
MIN_DISTINCT_ITEMS_PER_DAY = 10

for d in days:
    m.addConstr(gp.quicksum(fat[i] * x[i, d] for i in items) <= FAT_MAX, name=f"fat_max_day{d}")
    m.addConstr(gp.quicksum(sodium[i] * x[i, d] for i in items) <= SODIUM_MAX, name=f"sodium_max_day{d}")
    m.addConstr(gp.quicksum(protein[i] * x[i, d] for i in items) >= PROTEIN_MIN, name=f"protein_min_day{d}")
    m.addConstr(gp.quicksum(calories[i] * x[i, d] for i in items) >= CALORIE_MIN, name=f"calorie_min_day{d}")
    #m.addConstr(gp.quicksum(fiber[i] * x[i, d] for i in items) >= FIBER_MIN, name=f"fiber_min_day{d}")
    #m.addConstr(gp.quicksum(sugar[i] * x[i, d] for i in items) <= SUGAR_MAX, name=f"sugar_max_day{d}")

    # Limit total servings of each food per day
    m.addConstr(gp.quicksum(x[i, d] for i in items) <= MAX_ITEMS_PER_DAY, name=f"max_items_day{d}")

    
    # If y[i,d] = 0, then x[i,d] must be 0. And, if x[i,d] >= 1, then y[i,d] = 1
    for i in items:
        # Have x <= 2y so that we make the max servings of an item per day equal to 2
        m.addConstr(x[i, d] <= 2 * y[i, d], name=f"x_le_2y_{i}_day{d}")
        m.addConstr(x[i, d] >= y[i, d],   name=f"x_ge_y_{i}_day{d}")

    # Want atleast 10 different items per day for variety constraint
    m.addConstr(gp.quicksum(y[i, d] for i in items) >= MIN_DISTINCT_ITEMS_PER_DAY, name=f"min_distinct_items_day{d}")

# Each food can appear on at most 2 days in the week
# Do this outside of the loop since this is a constraint across the week instead of across a day
for i in items:
    m.addConstr(gp.quicksum(y[i, d] for d in days) <= 2, name=f"max_days_for_{i}")

m.optimize()

if m.Status == GRB.OPTIMAL:
    print("Total 7-day cost =", m.ObjVal)
    for d in days:
        print(f"\nDay {d + 1}:")
        for i in items:
            if x[i, d].X > 1e-6:
                print(f"  {i}: {int(round(x[i, d].X))}")
else:
    print("Status:", m.Status)


Set parameter Username
Set parameter LicenseID to value 2702155
Academic license - for non-commercial use only - expires 2026-09-02
Set parameter LogToConsole to value 0
Total 5-day cost = 69.22

Day 1:
  Cream of Mushroom Condensed Soup: 1
  FRENCH STYLE GREEN BEANS: 1
  Lite Nonfat Greek Yogurt: 1
  Lowfat Greek Yogurt: 1
  French green beans: 1
  Chili Hot Beans: 1
  Pork Flavored Gravy Mix: 2
  Green Lime Beans: 1
  Beans & Franks: 1
  Cream Style Golden Corn: 1

Day 2:
  Cream of Mushroom Condensed Soup: 1
  FRENCH STYLE GREEN BEANS: 1
  Lite Nonfat Greek Yogurt: 1
  Nonfat Greek Yogurt: 1
  French green beans: 1
  Chili Hot Beans: 1
  Pork Flavored Gravy Mix: 2
  Green Lime Beans: 1
  Beans & Franks: 1
  Cream Style Golden Corn: 1

Day 3:
  SUPER SWEET CORN: 1
  Chicken Pot Pie: 1
  Chunk Light Tuna in Water: 1
  Blended peach lowfat yogurt: 1
  Blended blueberry lowfat yogurt: 1
  Greek Nonfat Yogurt, Blueberry: 1
  Chunk Yellowfin Tuna In Water: 1
  Hot chili with beans: 1
  In